In [1]:
import bigframes
import bigframes.pandas as bpd
import bigframes.streaming as bs

In [2]:
bigframes.options._bigquery_options.project = "bigframes-load-testing"
job_id_prefix = "test_streaming_"
session = bpd.get_global_session()

/usr/local/google/home/garrettwu/src/bigframes/bigframes/pandas/__init__.py:853: DefaultLocationWarning: No explicit location is set, so using location US for the session.
  return global_session.get_global_session()


In [3]:
sdf = session.read_gbq_streaming("birds.penguins_bigtable_streaming")
type(sdf)

bigframes.streaming.StreamingDataFrame

In [4]:
sdf = sdf[["species", "island", "body_mass_g"]]
type(sdf)

bigframes.streaming.StreamingDataFrame

In [5]:
sdf = sdf[sdf["body_mass_g"] < 4000]
type(sdf)

bigframes.streaming.StreamingDataFrame

In [7]:
callable(getattr(sdf._df, "rename"))

True

In [4]:
sdf = sdf.rename(columns={"island": "rowkey"})
type(sdf)

bigframes.streaming.StreamingDataFrame

In [5]:
type(sdf)

bigframes.streaming.StreamingDataFrame

In [6]:
sdf

,species,rowkey,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Gentoo penguin (Pygoscelis papua),Biscoe,50.5,15.9,225,5400,MALE
1,Gentoo penguin (Pygoscelis papua),Biscoe,45.1,14.5,215,5000,FEMALE
2,Adelie Penguin (Pygoscelis adeliae),Torgersen,41.4,18.5,202,3875,MALE
3,Adelie Penguin (Pygoscelis adeliae),Torgersen,38.6,17.0,188,2900,FEMALE
4,Gentoo penguin (Pygoscelis papua),Biscoe,46.5,14.8,217,5200,FEMALE
5,Adelie Penguin (Pygoscelis adeliae),Biscoe,35.0,17.9,192,3725,FEMALE
6,Adelie Penguin (Pygoscelis adeliae),Dream,37.5,18.9,179,2975,<NA>
7,Gentoo penguin (Pygoscelis papua),Biscoe,42.0,13.5,210,4150,FEMALE
8,Gentoo penguin (Pygoscelis papua),Biscoe,48.5,14.1,220,5300,MALE
9,Adelie Penguin (Pygoscelis adeliae),Torgersen,45.8,18.9,197,4150,MALE


In [11]:
type(sdf)

bigframes.dataframe.DataFrame

In [7]:
sdf.sql

"WITH t0 AS (\n  SELECT\n    t4.`species`,\n    t4.`island`,\n    t4.`culmen_length_mm`,\n    t4.`culmen_depth_mm`,\n    t4.`flipper_length_mm`,\n    t4.`body_mass_g`,\n    t4.`sex`,\n    farm_fingerprint(\n      CONCAT(\n        CONCAT('\\\\', REPLACE(coalesce(t4.`species`, ''), '\\\\', '\\\\\\\\')),\n        CONCAT('\\\\', REPLACE(coalesce(t4.`island`, ''), '\\\\', '\\\\\\\\')),\n        CONCAT('\\\\', REPLACE(coalesce(CAST(t4.`culmen_length_mm` AS STRING), ''), '\\\\', '\\\\\\\\')),\n        CONCAT('\\\\', REPLACE(coalesce(CAST(t4.`culmen_depth_mm` AS STRING), ''), '\\\\', '\\\\\\\\')),\n        CONCAT('\\\\', REPLACE(coalesce(CAST(t4.`flipper_length_mm` AS STRING), ''), '\\\\', '\\\\\\\\')),\n        CONCAT('\\\\', REPLACE(coalesce(CAST(t4.`body_mass_g` AS STRING), ''), '\\\\', '\\\\\\\\')),\n        CONCAT('\\\\', REPLACE(coalesce(t4.`sex`, ''), '\\\\', '\\\\\\\\'))\n      )\n    ) AS `bigframes_ordering_2`,\n    farm_fingerprint(\n      CONCAT(\n        CONCAT(\n          CONCAT(

In [6]:
type(sdf)

bigframes.dataframe.DataFrame

In [10]:
job = sdf.to_bigtable(instance="streaming-testing-instance",
    table="garrettwu-no-col-family",
    service_account_email="streaming-testing-admin@bigframes-load-testing.iam.gserviceaccount.com",
    app_profile=None,
    truncate=True,
    overwrite=True,
    auto_create_column_families=True,
    bigtable_options={},
    job_id=None,
    job_id_prefix=job_id_prefix,)

/usr/local/google/home/garrettwu/src/bigframes/bigframes/streaming/__init__.py:187: PreviewWarning: The bigframes.streaming module is a preview feature, and subject to change.
  warnings.warn(


In [16]:
print(job.running())
print(job.error_result)

False
{'reason': 'stopped', 'debugInfo': '[CANCELLED] message=[CONTINUOUS_QUERY_RETRYABLE_ERROR] message=Retryable RPC error for continuous query. cause=com.google.net.rpc3.RpcException: CANCELLED debug= errorProto=code: "INTERNAL_ERROR"\nargument: "Error: 4345518"\n errorProto=code: "JOB_CANCELLED"\nargument: "User requested cancellation"\n\n\tat com.google.cloud.helix.common.Exceptions$Public.jobCancelled(Exceptions.java:947)\n\tat com.google.cloud.helix.common.Exceptions.fromCancelRequest(Exceptions.java:2101)\n\tat com.google.cloud.helix.common.CancelState.wrapException(CancelState.java:32)\n\tat com.google.cloud.helix.server.job.LocalJobController.handleJobDone(LocalJobController.java:590)\n\tat com.google.cloud.helix.server.job.LocalJobController.lambda$runThread$0(LocalJobController.java:452)\n\tat com.google.common.util.concurrent.CombinedFuture$CallableInterruptibleTask.runInterruptibly(CombinedFuture.java:198)\n\tat com.google.common.util.concurrent.InterruptibleTask.run(Inte

In [15]:
job.cancel()

True